In [1]:
from glob import glob
import os
import pickle
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aleksandr_Mishin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aleksandr_Mishin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Aleksandr_Mishin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Чтение данных в датафрейм:

In [2]:
PATH_TO_TEST_DATA = './test/'
PATH_TO_TRAIN_DATA = './train/'

In [3]:
def prepare_train_set(path_to_files):
    df = pd.DataFrame(columns=['text', 'target'])
    positive_files = glob(path_to_files + "pos/*.txt")
    negative_files = glob(path_to_files + "neg/*.txt")
    
    for file in tqdm(positive_files):
        with open(file, 'r', encoding="utf8") as content_file:
            content = content_file.read()
            df.loc[df.shape[0]] = [content, 1]
    for file in tqdm(negative_files):
        with open(file, 'r', encoding="utf8") as content_file:
            content = content_file.read()
            df.loc[df.shape[0]] = [content, 0]
    
    return df

In [4]:
df = prepare_train_set(PATH_TO_TRAIN_DATA)

HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))

Сериализуем датафрейм, чтобы не пересчитывать при перезапуске ядра.

In [5]:
with open ("./pickle/df.pkl", 'wb') as df_pkl:
    pickle.dump(df, df_pkl, protocol=2)

In [6]:
df = pickle.load(open("./pickle/df.pkl", "rb"))

In [7]:
pd.set_option('display.max_colwidth', -1)
df.head(3)

,text,target
0,"Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as ""Teachers"". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is ""Teachers"". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!",1
1,"Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they'll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it's like to be homeless? That is Goddard Bolt's lesson.<br /><br />Mel Brooks (who directs) who stars as Bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival (Jeffery Tambor) to see if he can live in the streets for thirty days without the luxuries; if Bolt succeeds, he can do what he wants with a future project of making more buildings. The bet's on where Bolt is thrown on the street with a bracelet on his leg to monitor his every move where he can't step off the sidewalk. He's given the nickname Pepto by a vagrant after it's written on his forehead where Bolt meets other characters including a woman by the name of Molly (Lesley Ann Warren) an ex-dancer who got divorce before losing her home, and her pals Sailor (Howard Morris) and Fumes (Teddy Wilson) who are already used to the streets. They're survivors. Bolt isn't. He's not used to reaching mutual agreements like he once did when being rich where it's fight or flight, kill or be killed.<br /><br />While the love connection between Molly and Bolt wasn't necessary to plot, I found ""Life Stinks"" to be one of Mel Brooks' observant films where prior to being a comedy, it shows a tender side compared to his slapstick work such as Blazing Saddles, Young Frankenstein, or Spaceballs for the matter, to show what it's like having something valuable before losing it the next day or on the other hand making a stupid bet like all rich people do when they don't know what to do with their money. Maybe they should give it to the homeless instead of using it like Monopoly money.<br /><br />Or maybe this film will inspire you to help others.",1
2,"Brilliant over-acting by Lesley Ann Warren. Best dramatic hobo lady I have ever seen, and love scenes in clothes warehouse are second to none. The corn on face is a classic, as good as anything in Blazing Saddles. The take on lawyers is also superb. After being accused of being a turncoat, selling out his boss, and being dishonest the lawyer of Pepto Bolt shrugs indifferently ""I'm a lawyer"" he says. Three funny words. Jeffrey Tambor, a favorite from the later Larry Sanders show, is fantastic here too as a mad millionaire who wants to crush the ghetto. His character is more malevolent than usual. The hospital scene, and the scene where the homeless invade a demolition site, are all-time classics. Look for the legs scene and the two big diggers fighting (one bleeds). This movie get

In [8]:
import string
df.text = df.text.apply(lambda x: x.lower())

tr = str.maketrans("", "", string.punctuation)
df.text = df.text.apply(lambda x: x.translate(tr))


In [9]:
df.head(1)

,text,target
0,bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell highs satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector im here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isnt,1


Убираем стоп слова

In [10]:
from nltk.corpus import stopwords

stoplist = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stoplist]))

In [11]:
df.head(3)

,text,target
0,bromwell high cartoon comedy ran time programs school life teachers 35 years teaching profession lead believe bromwell highs satire much closer reality teachers scramble survive financially insightful students see right pathetic teachers pomp pettiness whole situation remind schools knew students saw episode student repeatedly tried burn school immediately recalled high classic line inspector im sack one teachers student welcome bromwell high expect many adults age think bromwell high far fetched pity isnt,1
1,homelessness houselessness george carlin stated issue years never plan help street considered human everything going school work vote matter people think homeless lost cause worrying things racism war iraq pressuring kids succeed technology elections inflation worrying theyll next end streetsbr br given bet live streets month without luxuries home entertainment sets bathroom pictures wall computer everything treasure see like homeless goddard bolts lessonbr br mel brooks directs stars bolt plays rich man everything world deciding make bet sissy rival jeffery tambor see live streets thirty days without luxuries bolt succeeds wants future project making buildings bets bolt thrown street bracelet leg monitor every move cant step sidewalk hes given nickname pepto vagrant written forehead bolt meets characters including woman name molly lesley ann warren exdancer got divorce losing home pals sailor howard morris fumes teddy wilson already used streets theyre survivors bolt isnt hes used reaching mutual agreements like rich fight flight kill killedbr br love connection molly bolt wasnt necessary plot found life stinks one mel brooks observant films prior comedy shows tender side compared slapstick work blazing saddles young frankenstein spaceballs matter show like something valuable losing next day hand making stupid bet like rich people dont know money maybe give homeless instead using like monopoly moneybr br maybe film inspire help others,1
2,brilliant overacting lesley ann warren best dramatic hobo lady ever seen love scenes clothes warehouse second none corn face classic good anything blazing saddles take lawyers also superb accused turncoat selling boss dishonest lawyer pepto bolt shrugs indifferently im lawyer says three funny words jeffrey tambor favorite later larry sanders show fantastic mad millionaire wants crush ghetto character malevolent usual hospital scene scene homeless invade demolition site alltime classics look legs scene two big diggers fighting one bleeds movie gets better time see quite often,1


Применим PorterStemer на входные данные.

In [12]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

def stem_sentence(sentence):
    porter_stemmer = PorterStemmer()
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df['stem_text'] = df['text'].apply(stem_sentence)

Также применим лемматизацию в виде отдельного признака

In [13]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter

def get_pos(word):
    w_synsets = wordnet.synsets(word)

    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
    most_common_pos_list = pos_counts.most_common(3)
    return most_common_pos_list[0][0]

def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    tokens = sentence.split()
    lemmed_tokens = [lemmatizer.lemmatize(token, get_pos(token)) for token in tokens]
    return ' '.join(lemmed_tokens)

df['lemm_text'] = df['text'].apply(lemmatize_sentence)

In [14]:
df.head(1)

,text,target,stem_text,lemm_text
0,bromwell high cartoon comedy ran time programs school life teachers 35 years teaching profession lead believe bromwell highs satire much closer reality teachers scramble survive financially insightful students see right pathetic teachers pomp pettiness whole situation remind schools knew students saw episode student repeatedly tried burn school immediately recalled high classic line inspector im sack one teachers student welcome bromwell high expect many adults age think bromwell high far fetched pity isnt,1,bromwel high cartoon comedi ran time program school life teacher 35 year teach profess lead believ bromwel high satir much closer realiti teacher scrambl surviv financi insight student see right pathet teacher pomp petti whole situat remind school knew student saw episod student repeatedli tri burn school immedi recal high classic line inspector im sack one teacher student welcom bromwel high expect mani adult age think bromwel high far fetch piti isnt,bromwell high cartoon comedy run time program school life teacher 35 year teaching profession lead believe bromwell high satire much close reality teacher scramble survive financially insightful student see right pathetic teacher pomp pettiness whole situation remind school know student saw episode student repeatedly try burn school immediately recall high classic line inspector im sack one teacher student welcome bromwell high expect many adult age think bromwell high far fetch pity isnt


Применим аналогичные операции для отложенной выборки:

In [15]:
test_df = prepare_train_set(PATH_TO_TEST_DATA)
test_df.text = test_df.text.apply(lambda x: x.lower())
test_df.text = test_df.text.apply(lambda x: x.translate(tr))
test_df.text = test_df.text.apply(lambda x: ' '.join([word for word in x.split() if word not in stoplist]))
test_df['stem_text'] = test_df['text'].apply(stem_sentence)
test_df['lemm_text'] = test_df['text'].apply(lemmatize_sentence)

HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))

Применим bag-of-words для получения фич по токенам:

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.naive_bayes import MultinomialNB
import operator
import warnings
warnings.filterwarnings('ignore')

In [17]:
def get_top_words(vectorizer, model, n = 10):
    coefs_dict = dict(zip(vectorizer.get_feature_names(), model.coef_[0]))
    sorted_dict = sorted(coefs_dict.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_dict[:n]

def get_negative_words(vectorizer, model, n = 10):
    coefs_dict = dict(zip(vectorizer.get_feature_names(), model.coef_[0]))
    sorted_dict = sorted(coefs_dict.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_dict[-n:]

def train_and_predict_bow(estimator, count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,1)), column='text'):
    print("Model info: " + estimator.__class__.__name__ + " on dataset " + column + " preprocessed with " 
          + count_vectorizer.__class__.__name__)
    
    X_train = df[column]
    y_train = df['target']
    y_train = y_train.astype('bool')
    X_train_bow = count_vectorizer.fit_transform(X_train)
    estimator.fit(X_train_bow, y_train)  
    
    X_test_bow = count_vectorizer.transform(test_df[column])
    y_test = test_df.target
    y_test=y_test.astype('bool')


    print("accuracy: " + str(accuracy_score(y_test, estimator.predict(X_test_bow))))
    print("roc auc: " + str(roc_auc_score(y_test, estimator.predict(X_test_bow))))
    print('positive: ')
    print(get_top_words(count_vectorizer, estimator))
    print('negative: ')
    print(get_negative_words(count_vectorizer, estimator))
    print("\n")    

Начнём со стохастического градиентного спуска. Попробуем модель на:
- простом тексте без стоп слов и знаков препинаний;
- на нём же после стемминга;
- на нем же после лемматизации;

In [18]:
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='text')
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='stem_text')
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='lemm_text')

Model info: SGDClassifier on dataset text preprocessed with CountVectorizer
accuracy: 0.84548
roc auc: 0.84548
positive: 
[('710', 7.460376669656075), ('810', 4.523845427344654), ('flawless', 3.888919753331356), ('vengeance', 3.730188334828033), ('wonderfully', 3.6508226255763856), ('refreshing', 3.571456916324724), ('excellent', 3.4920912070730687), ('funniest', 3.492091207073062), ('perfectly', 3.4127254978214006), ('spectacular', 3.412725497821399)]
negative: 
[('lousy', -3.7301883348280476), ('mst3k', -3.8889197533313644), ('awful', -3.9682854625830286), ('310', -4.206382590338005), ('forgettable', -4.523845427344646), ('worst', -4.761942555099597), ('disappointment', -4.9206739736029474), ('poorly', -5.079405392106276), ('waste', -5.39686822911293), ('410', -6.58735386788782)]


Model info: SGDClassifier on dataset stem_text preprocessed with CountVectorizer
accuracy: 0.8356
roc auc: 0.8356
positive: 
[('710', 7.936570925166042), ('810', 5.476233938364572), ('refresh', 4.444479718

Как ни странно, стемминг дал чуть лучший результат чем лемматизация, последняя в свою очередь лучше простого текста. Правда, тут видна некоторая утечка данных - такие слова как '710', '810', '1010'  есть ни что иное, как оценки пользователей без символа '/'. В принципе их вполне можно считать как признаки, но для вычисления именно значящих слов, после первого теста моделей создадим ещё один вариант входных значений без цифр и чисел.

Добавим n-gram = 2

In [19]:
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='stem_text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='lemm_text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)))

Model info: SGDClassifier on dataset text preprocessed with CountVectorizer
accuracy: 0.8492
roc auc: 0.8491999999999998
positive: 
[('710', 7.6984737974110695), ('refreshing', 4.047651171834681), ('810', 3.8889197533313644), ('funniest', 3.809554044079708), ('well worth', 3.492091207073062), ('vengeance', 3.412725497821397), ('1010', 3.3333597885697435), ('wonderfully', 3.3333597885697395), ('subtle', 3.2539940793180846), ('ladder', 3.1746283700664213)]
negative: 
[('mst3k', -3.7301883348280382), ('forgettable', -3.888919753331368), ('awful', -3.9682854625830197), ('lacks', -4.206382590338007), ('310', -4.365114008841325), ('worst', -4.523845427344635), ('waste', -5.158771101357935), ('disappointment', -5.39686822911292), ('poorly', -5.634965356867892), ('410', -6.587353867887821)]


Model info: SGDClassifier on dataset stem_text preprocessed with CountVectorizer
accuracy: 0.85132
roc auc: 0.8513200000000001
positive: 
[('710', 7.381010960404424), ('810', 5.634965356867894), ('well wo

На биграммах лучше показал себя простой текст.

In [20]:
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,3)))
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='stem_text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,3)))
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='lemm_text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,3)))

Model info: SGDClassifier on dataset text preprocessed with CountVectorizer
accuracy: 0.85104
roc auc: 0.8510399999999999
positive: 
[('710', 7.3016452511527685), ('funniest', 4.047651171834699), ('well worth', 3.650822625576386), ('edie', 3.571456916324717), ('810', 3.4920912070730603), ('subtle', 3.412725497821411), ('rare', 3.4127254978214063), ('incredible', 3.4127254978213997), ('vengeance', 3.4127254978213983), ('refreshing', 3.3333597885697412)]
negative: 
[('baldwin', -3.650822625576381), ('mildly', -3.730188334828044), ('worst', -3.8095540440797055), ('310', -4.047651171834691), ('forgettable', -4.047651171834692), ('awful', -4.365114008841313), ('waste', -4.841308264351293), ('disappointment', -5.000039682854614), ('poorly', -5.634965356867895), ('410', -5.714331066119558)]


Model info: SGDClassifier on dataset stem_text preprocessed with CountVectorizer
accuracy: 0.85088
roc auc: 0.8508800000000001
positive: 
[('710', 7.777839506662717), ('810', 5.238136810609593), ('surpri

Триграммы не принесли новых значимых фич в топ-20, оставим для следующих моделей максимум в 2.

Попробуем байесовский классификатор:

In [21]:
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,1)), column='text')
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,1)), column='stem_text')
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,1)), column='lemm_text')

Model info: MultinomialNB on dataset text preprocessed with CountVectorizer
accuracy: 0.83832
roc auc: 0.8383200000000001
positive: 
[('br', -3.912733796080138), ('film', -4.252938887208872), ('movie', -4.329979539597176), ('one', -4.657868315181787), ('like', -5.061155397553605), ('good', -5.22718983837947), ('story', -5.358464176754282), ('great', -5.3886675523429854), ('time', -5.436338228977824), ('see', -5.462506684296146)]
negative: 
[('darkman', -14.136019629967338), ('delia', -14.136019629967338), ('hobgoblins', -14.136019629967338), ('kareena', -14.136019629967338), ('kornbluth', -14.136019629967338), ('sarne', -14.136019629967338), ('slater', -14.136019629967338), ('tashan', -14.136019629967338), ('tremors', -14.136019629967338), ('welch', -14.136019629967338)]


Model info: MultinomialNB on dataset stem_text preprocessed with CountVectorizer
accuracy: 0.8236
roc auc: 0.8236
positive: 
[('br', -3.9751872031626903), ('film', -4.0960526619146105), ('movi', -4.208899535600633), 

In [22]:
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)), column='text')
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)), column='stem_text')
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)), column='lemm_text')

Model info: MultinomialNB on dataset text preprocessed with CountVectorizer
accuracy: 0.84984
roc auc: 0.8498399999999999
positive: 
[('br', -3.9714711056751852), ('film', -4.311676196803919), ('movie', -4.388716849192223), ('one', -4.716605624776834), ('like', -5.119892707148653), ('good', -5.285927147974517), ('story', -5.417201486349329), ('great', -5.447404861938033), ('time', -5.495075538572872), ('see', -5.5212439938911935)]
negative: 
[('total waste', -13.50160975900244), ('uwe', -13.50160975900244), ('uwe boll', -13.50160975900244), ('wayans', -13.50160975900244), ('worst films', -13.50160975900244), ('acting horrible', -14.194756939562385), ('br 410', -14.194756939562385), ('hours life', -14.194756939562385), ('prom night', -14.194756939562385), ('worst acting', -14.194756939562385)]


Model info: MultinomialNB on dataset stem_text preprocessed with CountVectorizer
accuracy: 0.84372
roc auc: 0.84372
positive: 
[('br', -4.058544702002372), ('film', -4.179410160754292), ('movi',

Тут мы видим довольно много распространенных слов, но это происходит во многом из-за их высокой априорной вероятности. В негативных значения поинтереснее(особенно uwe и uwe boll, а также wayans - плохие режиссеры становятся именем нарицательным).

In [23]:
train_and_predict_bow(estimator=svm.LinearSVC(), column='text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=svm.LinearSVC(), column='stem_text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=svm.LinearSVC(), column='lemm_text', count_vectorizer=CountVectorizer(max_features=10000, ngram_range=(1,2)))

Model info: LinearSVC on dataset text preprocessed with CountVectorizer
accuracy: 0.83704
roc auc: 0.83704
positive: 
[('710', 1.8118395485843473), ('refreshing', 1.1422228019450797), ('well worth', 1.0982276008608238), ('even better', 1.0499683521851582), ('able see', 1.0441149755981602), ('surprisingly good', 1.0347886258685592), ('pickup', 1.0265251048854012), ('michael jackson', 0.9992343560880729), ('flawless', 0.9953712049967014), ('respectable', 0.9677207265278389)]
negative: 
[('one worst', -0.9950760137023653), ('poorly', -1.0512802299856556), ('forgettable', -1.0744630427075774), ('baldwin', -1.0889630415990892), ('refer', -1.1240466851052238), ('mst3k', -1.155794962798977), ('disappointment', -1.1714271923924522), ('210', -1.2547420230667088), ('310', -1.2978391307659871), ('410', -1.8871924506396707)]


Model info: LinearSVC on dataset stem_text preprocessed with CountVectorizer
accuracy: 0.83064
roc auc: 0.83064
positive: 
[('710', 2.0195486641242395), ('driven', 1.1940194

Повторим все те же операции, но теперь с TF-IDF вместо Bag-of-Words. Будем использовать сразу биграммы.

In [24]:
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='text', count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='stem_text', count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='lemm_text', count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)))

train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)), column='text')
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)), column='stem_text')
train_and_predict_bow(estimator=MultinomialNB(), count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)), column='lemm_text')

train_and_predict_bow(estimator=svm.LinearSVC(), column='text', count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=svm.LinearSVC(), column='stem_text', count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)))
train_and_predict_bow(estimator=svm.LinearSVC(), column='lemm_text', count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)))

Model info: SGDClassifier on dataset text preprocessed with TfidfVectorizer
accuracy: 0.88768
roc auc: 0.88768
positive: 
[('excellent', 3.6586176983394547), ('great', 3.564119687567475), ('perfect', 2.885532229361667), ('710', 2.8483684441392674), ('wonderful', 2.8036484400632617), ('best', 2.7408331783498254), ('amazing', 2.5992493743224943), ('favorite', 2.555094724974988), ('fun', 2.3540796983544943), ('today', 2.128088868744667)]
negative: 
[('worse', -2.839250731208912), ('dull', -2.900881420763492), ('terrible', -2.916377481400749), ('nothing', -3.0432219121946837), ('waste', -3.4676889339485584), ('poor', -3.48932384321161), ('boring', -3.497366351256867), ('awful', -4.056783715704232), ('bad', -4.194764153509881), ('worst', -4.934198291219931)]


Model info: SGDClassifier on dataset stem_text preprocessed with TfidfVectorizer
accuracy: 0.88508
roc auc: 0.88508
positive: 
[('great', 3.766511227829611), ('excel', 3.7658930538468085), ('perfect', 2.991291215480247), ('enjoy', 2.8


Использование TF-IDF дало прирост в точности примерно на 4%. Попробуем ещё настроить регуляризацию для SVM.

In [25]:
from sklearn.model_selection import GridSearchCV

def train_and_predict_grid(estimator=svm.SVC(), count_vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,2)), column='text'):
    print("Model info: " + estimator.__class__.__name__ + " on dataset " + column + " preprocessed with " 
          + count_vectorizer.__class__.__name__)
    
    X_train = df[column]
    y_train = df['target']
    y_train = y_train.astype('bool')
    X_train_bow = count_vectorizer.fit_transform(X_train)
    
    parameters = {'kernel':('linear', 'rbf'), 'C':[10, 100, 1000]}
    grid = GridSearchCV(estimator, parameters, cv=3, n_jobs=-1, verbose=True)
    grid.fit(X_train_bow, y_train)  
    
    
    X_test_bow = count_vectorizer.transform(test_df[column])
    y_test = test_df.target
    y_test=y_test.astype('bool')

    print('best parameters: ')
    print(grid.best_estimator_.get_params())
    print("accuracy: " + str(accuracy_score(y_test, grid.best_estimator_.predict(X_test_bow))))
    print("roc auc: " + str(roc_auc_score(y_test, grid.best_estimator_.predict(X_test_bow))))
    print("\n")   

In [26]:
train_and_predict_grid(estimator=svm.SVC(), column='text', count_vectorizer=TfidfVectorizer(max_features=1000, ngram_range=(1,2)))
# train_and_predict_grid(estimator=svm.SVC(), column='stem_text', count_vectorizer=TfidfVectorizer(max_features=1000, ngram_range=(1,2)))
# train_and_predict_grid(estimator=svm.SVC(), column='lemm_text', count_vectorizer=TfidfVectorizer(max_features=1000, ngram_range=(1,2)))

Model info: SVC on dataset text preprocessed with TfidfVectorizer
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 119.3min finished


best parameters: 
{'C': 100, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
accuracy: 0.8586
roc auc: 0.8586




В соседнем ноутбуке все те же операции выполнены с фильтраций числовых символов.
Из общих итогов:
- tf-idf показал себя в задаче лучше чем bag-of-words. Общий прирост по обоим метрикам - 4%.
- при использование ngram=1 стеминг > лемматизация > простой текст. Однако, при использовании биграм и триграм простой текст начал показывать такой же или чуть лучший результат.
- из моделей лучшие показатели выглядят с небольшой разницей так: SGD > SVM > NB. Но NB и SGD показали гораздо более высокую скорость обучения.
- выключение числовых данных привело к небольшому ухудшению ~0-1%